In [270]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import requests, re
from bs4 import BeautifulSoup
import pandas as pd
import unidecode

from tqdm import tqdm_notebook

In [128]:
def get_AOTY():

    df = pd.DataFrame(columns =['artist', 'album', 'rating'])

    for page in tqdm_notebook(range(1, 120+1)):
        url = "https://www.albumoftheyear.org/releases/" + str(page) + "/"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        tmptag = soup.select("#centerContent > div.flexContainer > div.wideLeft .albumBlock")

        for i in range(len(tmptag)):
            source = tmptag[i]

            artist = source.select(".artistTitle")[0].get_text()
            album = source.select(".albumTitle")[0].get_text()

            try:
                rating = source.select(".ratingRow .ratingBlock .rating")[0].get_text()
            except IndexError:
                rating = 0

            data = {
                'artist' : artist,
                'album' : album,
                'rating' : rating,
            }

            df.loc[len(df)] = data
    
    return df

In [129]:
rating_AOTY = get_AOTY()

HBox(children=(IntProgress(value=0, max=120), HTML(value='')))

In [231]:
print(rating_AOTY.shape)
rating_AOTY.head()

(8945, 3)


,artist,album,rating
0,Janelle Monáe,Dirty Computer,87
1,Post Malone,Beerbongs & Bentleys,53
2,Grouper,Grid of Points,79
3,Speedy Ortiz,Twerp Verse,76
4,Anne-Marie,Speak Your Mind,55


In [265]:
def decodeuni(str):
    return unidecode.unidecode(str)

In [268]:
rating_AOTY['artist'] = rating_AOTY['artist'].apply(decodeuni)
rating_AOTY['album'] = rating_AOTY['album'].apply(decodeuni)
rating_AOTY['artist'] = rating_AOTY['artist'].apply(lambda x : x.replace(",", ""))
rating_AOTY['album'] = rating_AOTY['album'].apply(lambda x : x.replace(",", ""))

In [273]:
rating_AOTY.to_csv("../data/rating_AOTY.csv", index=False)

In [274]:
df = pd.read_csv("../data/rating_AOTY.csv")

In [275]:
print(df.shape)
df.head()

(8945, 3)


,artist,album,rating
0,Janelle Monae,Dirty Computer,87
1,Post Malone,Beerbongs & Bentleys,53
2,Grouper,Grid of Points,79
3,Speedy Ortiz,Twerp Verse,76
4,Anne-Marie,Speak Your Mind,55


In [276]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [277]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [278]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [280]:
df.to_sql(name="rating_AOTY", con=engine, if_exists='replace')